In [1]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()) + '/source')

import numpy as np
import matplotlib.pyplot as plt
import SolveLinSys
from utilities import compute_derivatives_3d

Equation:

$$
0 = \max_{e} \min_{h_2} b\left[\delta \eta \log e - \delta(1-\eta) \color{red}{\gamma_1} y z_2 -\frac{\gamma_2}{2}\beta_f^2 z_2^2y^2  + \xi_m\frac{(h_2)^2}{2}\right] -\ell e + \frac{\partial \phi}{\partial y}(y, b, z_2;\ell) e - \frac{\partial \phi}{\partial b}(y, b, z_2;\ell) \delta b + \left[\frac{\partial \phi}{\partial z_2}(y, b, z_2;\ell)\right]\left[-\rho (z_2 -\mu_2) + \sqrt{z_2} \sigma_2 h_2\right] + \left[\frac{\partial^2 \phi}{\partial(z_2)^2}(y,b, z_2;\ell)\right]\left(\frac{z_2|\sigma_2|^2}{2}\right)
$$

where

\begin{align*}
\color{red}{\gamma_1} &= 0.00017675\\
\gamma_2 &= 0.0044\\
\beta_f & = 1.86/1000
\end{align*}

FOC of $h_2$ gives : 

$$
h_2^* = -\frac{\left[\frac{\partial \phi}{\partial z_2}(y, b, z_2;\ell)\right]\sqrt{z_2}\sigma_2}{b\xi_m}
$$

FOC of $e$ gives :

$$
e^* = \frac{b \delta \eta}{\ell - \frac{\partial \phi}{\partial y}(y, b, z_2; \ell)}
$$

In [11]:
def false_transient_one_iteration(stateSpace, A, B_z, B_b, B_y, C_zz, C_bb, C_yy, D, v0, ε=.3, tol=-10):
    A = A.reshape(-1, 1, order='F')
    B = np.hstack([B_z.reshape(-1, 1, order='F'), B_b.reshape(-1, 1, order='F'), B_y.reshape(-1, 1, order='F')])
    C = np.hstack([C_zz.reshape(-1, 1, order='F'), C_bb.reshape(-1, 1, order='F'), C_yy.reshape(-1, 1, order='F')])
    D = D.reshape(-1, 1, order='F')
    v0 = v0.reshape(-1, 1, order='F')
    out = SolveLinSys.solveFT(stateSpace, A, B, C, D, v0, ε, tol)
    return out


def false_transient(z_grid, b_grid, y_grid, ℓ, model_paras=(), v0=None, ϵ=.5, tol=1e-8, max_iter=10_000):
    τ, η, ξ_m, δ, η, μ_2, ρ, σ_2 = model_paras
    β_f = 1.86/1000
    γ_2 = 2*0.0022

    Δ_z = z_grid[1] - z_grid[0]
    Δ_b = b_grid[1] - b_grid[0]
    Δ_y = y_grid[1] - y_grid[0]
    (z_mat, b_mat, y_mat) = np.meshgrid(z_grid, b_grid, y_grid, indexing = 'ij')
    stateSpace = np.hstack([z_mat.reshape(-1, 1, order='F'), b_mat.reshape(-1, 1, order='F'), y_mat.reshape(-1, 1, order='F')])

    if v0 is None:
        v0 = np.zeros_like(z_mat)

    count = 1
    error = 1.
    
#     e_old = 0. # smooth
#     h_old = 0. # smooth
    
    while error > tol and count < max_iter:
        v_old = v0.copy()
        v0_dz = compute_derivatives_3d(v0, 0, 1, Δ_z)
        v0_dzz = compute_derivatives_3d(v0, 0, 2, Δ_z)
        v0_db = compute_derivatives_3d(v0, 1, 1, Δ_b)
        v0_dy = compute_derivatives_3d(v0, 2, 1, Δ_y)

        e = b_mat*δ*η / (ℓ-v0_dy)
        e[e<=0] = 1e-5
        h = -v0_dz*np.sqrt(z_mat)*σ_2 / (b_mat*ξ_m)
        
#         e = e*0.1 + e_old*0.9 # smooth
#         h = h*0.1 + h_old*0.9 # smooth
        
        A = np.zeros_like(z_mat)
        B_z = -ρ*(z_mat - μ_2) + np.sqrt(z_mat)*σ_2*h
        B_b = -(δ*b_mat)
        B_y = e
        C_zz = z_mat*σ_2**2/2
        C_bb = np.zeros_like(z_mat)
        C_yy = np.zeros_like(z_mat)
        D = b_mat*(δ*η*np.log(e) - δ*(1-η)*y_mat*z_mat - γ_2*β_f**2*z_mat**2*y_mat**2/2 + ξ_m*h**2/2) - ℓ*e

        res = false_transient_one_iteration(stateSpace, A, B_z, B_b, B_y, C_zz, C_bb, C_yy, D, v0, ϵ) # CG tol = 1e-10
        v0 = res[2].reshape(v0.shape, order = "F")

        rhs_error = A*v0 + B_z*v0_dz + B_b*v0_db + B_y*v0_dy + C_zz*v0_dzz + D
        rhs_error = np.max(abs(rhs_error))
        lhs_error = np.max(abs((v0 - v_old)/ϵ))
        error = lhs_error
        
#         e_old = e.copy() # smooth
#         h_old = h.copy() # smooth
        
        print("Iteration %s: LHS Error: %s; RHS Error %s" % (count, lhs_error, rhs_error))
        count += 1
    print("Iteration %s: LHS Error: %s; RHS Error %s" % (count, lhs_error, rhs_error))
    return v0, e

In [16]:
# Model parameters
median = 1.75/1000
γ = .018
τ = median * γ
η = .032
ξ_m = .00256
δ = .01
η = .032

μ_2 = 1.
ρ = 0.99
σ_2 = np.sqrt((.21)**2*2*ρ/μ_2) # Match moments, using 100 year's std
# σ_2 = 0.004

model_paras = (τ, η, ξ_m, δ, η, μ_2, ρ, σ_2)

# Grid setting
n_z = 20
z_min = 0.9
z_max = 1.1
z_grid = np.linspace(z_min, z_max, n_z)

n_b = 20
b_min = 2e-2
b_max = 1.
b_grid = np.linspace(b_min, b_max, n_b)

n_y = 40
y_min = 1e-2
y_max = 3000.
y_grid = np.linspace(y_min, y_max, n_y)

# ℓ setting
log_ell_grid = np.linspace(-13, -5, 200)
ℓ_step = 1e-7

In [17]:
σ_2

0.2954961928688761

In [14]:
# ψs = []
# ψs_next = []
# ψ_guess = np.load('phi_guess.npy')

# for i, ℓ in enumerate(np.exp(log_ell_grid)):
#     print(i)
#     max_iter = 2000
#     ψ = false_transient(z_grid, b_grid, y_grid, ℓ, model_paras=model_paras, v0=ψ_guess, ϵ=.5, tol=1e-8, max_iter=max_iter)
#     ψ_next = false_transient(z_grid, b_grid, y_grid, ℓ+ℓ_step, model_paras=model_paras, v0=ψ_guess, ϵ=.5, tol=1e-8, max_iter=max_iter)
#     ψs.append(ψ)
#     ψs_next.append(ψ_next)

In [15]:
ψ, e = false_transient(z_grid, b_grid, y_grid, 1e-7, model_paras=model_paras, v0=None, ϵ=.5, tol=1e-8, max_iter=10)

Iteration 1: LHS Error: 12.652932071105354; RHS Error 32.02462240685159
Iteration 2: LHS Error: 59.38041936658091; RHS Error 29365.906529742795
Iteration 3: LHS Error: 19.75943595785743; RHS Error 7323.36834993054
Iteration 4: LHS Error: 20.82153539715553; RHS Error 1834.1133365537303
Iteration 5: LHS Error: 24.859041525790825; RHS Error 473.10935214986984
Iteration 6: LHS Error: 27.25440900563641; RHS Error 10112.391467396357
Iteration 7: LHS Error: 34.80328761550216; RHS Error 11865.881560825612
Iteration 8: LHS Error: 40.93275944993485; RHS Error 35937.07931708125
Iteration 9: LHS Error: 75.14678803106492; RHS Error 1844404.4157873264
Iteration 10: LHS Error: 75.14678803106492; RHS Error 1844404.4157873264


array([[[ -0.39946147,  -0.48093477,  -0.17055966, ...,  -1.5418252 ,
          -1.76839654,  -1.56558802],
        [ -0.10837369,  -0.20611687,  -0.32574031, ...,  -6.88207094,
          -5.82096958,  -1.0233314 ],
        [ -0.313319  ,  -0.36013671,  -0.60015782, ..., -10.10550676,
          -7.95255048,  -8.30490852],
        ...,
        [-13.35625494, -16.4481433 , -12.4314321 , ..., -63.82795087,
         -48.07246279, -31.01113452],
        [ -3.10065803,  -8.33036227,  -7.16499431, ..., -64.64641664,
         -46.37429319, -31.0338881 ],
        [ -5.43458264,  -8.19199159,  -8.83924226, ..., -61.94491758,
         -44.39954001, -33.13498337]],

       [[ -0.40056791,  -0.48143628,  -0.16673048, ...,  -1.54164452,
          -1.76857188,  -1.56569964],
        [ -0.10861638,  -0.20574751,  -0.32565441, ...,  -6.88174576,
          -5.81990852,  -1.0235494 ],
        [ -0.3129983 ,  -0.35987403,  -0.60034548, ..., -10.1047676 ,
          -7.9516797 ,  -8.30552018],
        ...,


In [ ]:
# np.save('phi_guess.npy', ψ)

In [ ]:
# fig = go.Figure(data=[go.Surface(z=ψ[10,:,:])])

# fig.update_layout(title='Mt Bruno Elevation', autosize=False,
#                   width=500, height=500,
#                   margin=dict(l=65, r=50, b=65, t=90))

# fig.show()

plt.figure()
plt.plot(y_grid, ψ[10,10,:])
plt.title(r'Figure 3: $\phi(y,b,z_2)$ when $z_2=1.$, $b=0.5$ ')
plt.xlabel('y')
plt.ylabel('ϕ')
plt.savefig('fig_3.pdf')

plt.figure()
plt.plot(y_grid, ψ[10,10,:])
plt.title(r'Figure 2: $\phi(y,b,z_2)$ when $z_2=1.$, $b=0.01$ ')
plt.xlabel('y')
plt.ylabel('ϕ')
# plt.savefig('fig_2.pdf')

In [ ]:
plt.figure()
plt.plot(y_grid, ψ[10,20,:])
plt.title(r'Figure 1: $\phi(y,b,z_2)$ when $z_2=1.$, $b=0.5$ ')
plt.xlabel('y')
plt.ylabel('ϕ')
plt.savefig('fig_3.pdf')

In [ ]:
from numba import njit
@njit
def simulate_z(μ, ρ, σ, T):
    Ws = np.random.normal(0., 1., T)
    Zs = np.zeros(T)
    Zs[0] = μ
    for i in range(1, T):
        Zs[i] = Zs[i-1] - ρ*(Zs[i-1]-μ) + np.sqrt(Zs[i-1])*σ*Ws[i]
    return Zs


def simulate_z_path(μ, ρ, σ, T, dt=1./365):
    time_periods = int(T/dt)
    time_path = np.arange(dt, T+dt, dt)
    Ws = np.random.normal(0., dt, time_periods)
    Zs = np.zeros(time_periods)
    Zs[0] = μ
    for i in range(1, time_periods):
        Zs[i] = Zs[i-1] - ρ*(Zs[i-1]-μ)*dt + np.sqrt(Zs[i-1])*σ*Ws[i]
    return Zs, time_path

In [ ]:
res = simulate_z(μ_2, .99, 0.004, 1000)
# plt.hist(res, bins=2000, label=r'Var(Z) = .004')
# plt.title(r'Figure 1: $\mu_2 = 1$, $\rho = .99$, $\sigma_2 = .004$')
# plt.legend()
# plt.savefig('fig_1.pdf')
# plt.show()

In [ ]:
res_z, res_t = simulate_z_path(μ_2, .99, 0.004, 200, 1./12)
plt.plot(res_t, res_z)
plt.title(r'$Z_t$, path 4')
plt.xlabel('Year')
plt.savefig('path_4.pdf')
# res = simulate_z(μ_2, .99, 0.004, 1000)
# plt.plot(res[:200])
# res = simulate_z(μ_2, .99, 0.004, 1000)
# plt.plot(res[:200])

In [ ]:
np.std(res)*5

In [ ]:
np.percentile(res, 1)

In [ ]:
res = simulate_z(μ_2, 1e-8, 2.97e-5, 100_000_000)

In [ ]:
plt.hist(res, bins=2000, label=r'Var(Z) = 0.21')
plt.title(r'Figure 2: $\mu_2 = 1$, $\rho = 10^{-8}$, $\sigma_2 = 3*10^{-5}$')
plt.legend()
plt.savefig('fig_2.pdf')
plt.show()